# Part 1: Distributed data parallel MNIST training with PyTorch and SageMaker distributed

## Background
[Amazon SageMaker's distributed library](https://docs.aws.amazon.com/sagemaker/latest/dg/distributed-training.html) can be used to train deep learning models faster and cheaper. The [data parallel](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel.html) feature in this library is a distributed data parallel training framework for PyTorch, TensorFlow, and MXNet. This notebook demonstrates how to use the SageMaker distributed data library to train a PyTorch model using the MNIST dataset.

This notebook example shows how to use `smdistributed.dataparallel` with PyTorch in SageMaker using MNIST dataset.

For more information:

1. [SageMaker distributed data parallel PyTorch API Specification](https://sagemaker.readthedocs.io/en/stable/api/training/smd_data_parallel_pytorch.html)
1. [Getting started with SageMaker distributed data parallel](https://sagemaker.readthedocs.io/en/stable/api/training/smd_data_parallel.html)
1. [PyTorch in SageMaker](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html)

### Dataset
This example uses the MNIST dataset. MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits).

**NOTE:** This example requires SageMaker Python SDK v2.X.

In [17]:
pip install sagemaker --upgrade

Requirement already up-to-date: sagemaker in /home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (2.45.0)
Note: you may need to restart the kernel to use updated packages.


### SageMaker role

The following code cell defines `role` which is the IAM role ARN used to create and run SageMaker training and hosting jobs. This is the same IAM role used to create this SageMaker Notebook instance. 

`role` must have permission to create a SageMaker training job and launch an endpoint to host a model. For granular policies you can use to grant these permissions, see [Amazon SageMaker Roles](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html). If you do not require fine-tuned permissions for this demo, you can used the IAM managed policy AmazonSageMakerFullAccess to complete this demo. 

In [18]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
role_name = role.split(["/"][-1])
print(f"The Amazon Resource Name (ARN) of the role used for this demo is: {role}")
print(f"The name of the role used for this demo is: {role_name[-1]}")

The Amazon Resource Name (ARN) of the role used for this demo is: arn:aws:iam::688520471316:role/hongshan-sagemaker-experiment
The name of the role used for this demo is: hongshan-sagemaker-experiment


To verify that the role above has required permissions:

1. Go to the IAM console: https://console.aws.amazon.com/iam/home.
2. Select **Roles**.
3. Enter the role name in the search box to search for that role. 
4. Select the role.
5. Use the **Permissions** tab to verify this role has required permissions attached.

## Model training with SageMaker distributed data parallel

### Training script

The MNIST dataset is downloaded using the `torchvision.datasets` PyTorch module; you can see how this is implemented in the `train_pytorch_smdataparallel_mnist.py` training script that is printed out in the next cell.

The training script provides the code you need for distributed data parallel (DDP) training using SageMaker's distributed data parallel library (`smdistributed.dataparallel`). The training script is very similar to a PyTorch training script you might run outside of SageMaker, but modified to run with the `smdistributed.dataparallel` library. This library's PyTorch client provides an alternative to PyTorch's native DDP. 

For details about how to use `smdistributed.dataparallel`'s DDP in your native PyTorch script, see the [Modify a PyTorch Training Script Using SMD Data Parallel](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel-modify-sdp.html#data-parallel-modify-sdp-pt).

In [19]:
!pygmentize code/train_pytorch_smdataparallel_mnist.py

# Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License"). You
# may not use this file except in compliance with the License. A copy of
# the License is located at
#
#     http://aws.amazon.com/apache2.0/
#
# or in the "license" file accompanying this file. This file is
# distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF
# ANY KIND, either express or implied. See the License for the specific
# language governing permissions and limitations under the License.

from __future__ import print_function

import argparse
import os
import time

import smdistributed.dataparallel.torch.distributed as dist
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Network definition
from model_def import Net

# Import SMDataParallel PyTorch Modules
from smdistributed.dataparallel.torch.parallel.distributed import DistributedDataParallel as DDP
from torch.opti

### Estimator function options

In the following code block, you can update the estimator function to use a different instance type, instance count, and distrubtion strategy. You're also passing in the training script you reviewed in the previous cell to this estimator.

**Instance types**

`smdistributed.dataparallel` supports model training on SageMaker with the following instance types only.  For best performance, it is recommended you use an instance type that supports Amazon Elastic Fabric Adapter (ml.p3dn.24xlarge and ml.p4d.24xlarge).

1. ml.p3.16xlarge
1. ml.p3dn.24xlarge [Recommended]
1. ml.p4d.24xlarge [Recommended]

**Instance count**

To get the best performance and the most out of `smdistributed.dataparallel`, you should use at least 2 instances, but you can also use 1 for testing this example.

**Distribution strategy**

Note that to use DDP mode, you update the the `distribution` strategy, and set it to use `smdistributed dataparallel`. 

In [20]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    base_job_name="pytorch-smdataparallel-mnist",
    source_dir="code",
    entry_point="train_pytorch_smdataparallel_mnist.py",
    role=role,
    framework_version="1.8.1",
    py_version="py36",
    # For training with multinode distributed training, set this count. Example: 2
    instance_count=1,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type="ml.p3dn.24xlarge",
    sagemaker_session=sagemaker_session,
    # Training using SMDataParallel Distributed Training Framework
    distribution={"smdistributed": {"dataparallel": {"enabled": True}}},
    debugger_hook_config=False,
)

In [21]:
estimator.fit()

2021-06-14 23:25:30 Starting - Starting the training job...
2021-06-14 23:25:59 Starting - Launching requested ML instancesProfilerReport-1623713130: InProgress
......
2021-06-14 23:26:59 Starting - Preparing the instances for training......
2021-06-14 23:27:59 Downloading - Downloading input data
2021-06-14 23:27:59 Training - Downloading the training image.........................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-06-14 23:31:59,550 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-06-14 23:31:59,625 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-06-14 23:32:01,048 sagemaker_pytorch_container.training INFO     Invoking SMDataParallel
2021-06-14 23:32:01,048 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-06-14 23:32:01,512 sagemaker-training-toolkit INFO     Starting MPI run as 

## Next steps

Now that you have a trained model, you can deploy an endpoint to host the model. After you deploy the endpoint, you can then test it with inference requests. The following cell will store the model_data variable to be used with the inference notebook.

In [22]:
model_data = estimator.model_data
print("Storing {} as model_data".format(model_data))
%store model_data

Storing s3://sagemaker-us-west-2-688520471316/pytorch-smdataparallel-mnist-2021-06-14-23-25-29-876/output/model.tar.gz as model_data
Stored 'model_data' (str)


In [25]:
!aws s3 ls s3://sagemaker-us-west-2-688520471316/pytorch-smdataparallel-mnist-2021-06-14-23-25-29-876/output/

2021-06-14 23:34:59    4453403 model.tar.gz
